## 라이브러리 임포트_20221114
- 실습에 필요한 라이브러리를 임포트한다.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 불러오기 (학습 데이터, 테스트 데이터)_20221114
- 데이터 분석 단계에서 생성한 농구 포지션 예측하기의 학습 데이터 및 테스트 데이터를 로드합니다.

In [3]:
train = pd.read_csv("data/csv/basketball_train.csv")
test = pd.read_csv("data/csv/basketball_test.csv")

In [7]:
print(train.shape[0]) # 행의 값만 확인 # 훈련용 데이터 셋 갯수
print(train.shape[1]) # feature(특징, 열)의 값만 확인 # 훈련용 데이터 셋 갯수

print(test.shape[0]) # 테스트 데이터 셋 행 갯수
print(test.shape[1]) # 테스트 데이터 셋 열 갯수

80
5
20
5


## 최적의 k값 찾기_20221114